참고문헌 : Pytel 외 저, 이주성 외 역, 재료역학, 2판, 한티미디어, 2013.<br>Ref: Pytel, Kiusalaas, Sharma, Mechanics of Materials, 2nd Ed., Cengage Learning, 2013.

`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)<br>
Bring in `module`'s that would expand features of `python`. (This file may use just some of them.)

In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import sympy.plotting as splot
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비

## 예제 05.005

### 문제에서 주어진 변수

#### 각 구간 길이

In [ ]:
L_AB_m, L_BD_m = sy.symbols('L_{AB}[m], L_{BD}[m]', positive=True)

L_AD_m = L_AB_m + L_BD_m

s_d = {L_AB_m: 4, 
       L_BD_m: 2}

#### 재료

In [ ]:
sigma_allowed_Pa = sy.Symbol('\sigma_{allowed}[Pa]', positive=True)
s_d[sigma_allowed_Pa] = 120e6
sigma_allowed_Pa

#### 하중

In [ ]:
P_B_N = sy.Symbol('P_{B}[N]', real=True)
s_d[P_B_N] = -45e3

### A, B, C, D 지점의 x 좌표
A 점에서 x=0 으로 한다.

In [ ]:
x_A_m = 0
x_B_m = float(x_A_m + L_AB_m.subs(s_d))
x_D_m = float(x_B_m + L_BD_m.subs(s_d))

### 반력

단순지지로 A D 점에 수직 방향 반력이 가해짐

#### 수직 반력

In [ ]:
R_A_N, R_D_N = sy.symbols('R_{A}[N], R_{D}[N]', real=True)

In [ ]:
vf_eq = sy.Eq(R_A_N + R_D_N + P_B_N, 0)
vf_eq

#### A점 중심 모멘트

In [ ]:
moment_A_eq = sy.Eq(P_B_N * L_AB_m + R_D_N * L_AD_m, 0)
moment_A_eq

#### 두 식을 연립하여 풂

In [ ]:
R_sol_dict = sy.solve((vf_eq, moment_A_eq), (R_A_N, R_D_N))
R_sol_dict

### 전단력선도

In [ ]:
x = sy.Symbol('x', nonnegative=True)

In [ ]:
V_AD_N = R_A_N + (R_A_N + P_B_N) * sy.Heaviside(x - L_AB_m)

반력 계산 결과를 반영하면 다음과 같다.

In [ ]:
V_AD_N.subs(R_sol_dict)

In [ ]:
V_AD_N.subs(R_sol_dict).subs(s_d)

선도로 표시해 보자.

In [ ]:
splot.plot(
    sy.N(V_AD_N.subs(R_sol_dict).subs(s_d)), 
    (x, 0, float(L_AD_m.subs(s_d))), 
    ylabel='V(N)'
)

### 굽힘모멘트선도

In [ ]:
M_AB_Nm = R_A_N * x
M_BD_Nm = P_B_N * (x - L_AB_m)

In [ ]:
M_AD_Nm = M_AB_Nm + M_BD_Nm * sy.Heaviside(x - L_AB_m)

In [ ]:
M_AD_Nm

In [ ]:
M_AD_Nm.subs(R_sol_dict)

In [ ]:
splot.plot((M_AD_Nm.subs(R_sol_dict).subs(s_d), (x, 0, x_D_m)), ylabel='M(Nm)')

#### 최대 모멘트

In [ ]:
M_max_Nm = M_AD_Nm.subs(R_sol_dict).subs(x, L_AB_m)
M_max_Nm

### 단면 계수

In [ ]:
section_modulus_mm3 = sy.Symbol('section_{modulus}[mm^{3}]', positive=True)
section_modulus_m3 = section_modulus_mm3 * 1e-3**3

In [ ]:
max_stress_eq = (abs(M_max_Nm / section_modulus_m3) < sigma_allowed_Pa)
max_stress_eq

In [ ]:
sm3_sol_list = sy.solve(max_stress_eq.subs(R_sol_dict).subs(s_d), section_modulus_m3)
sm3_sol_list

### 선정 단면 분포 하중 포함

#### 자중에 의한 분포하중

선정 단면 길이당 질량

In [ ]:
mw_kg_m = sy.Symbol('mw_{kg}[m]', positive=True)
s_d[mw_kg_m] = 38.7

중력가속도

In [ ]:
g_m_s2 = sy.Symbol('g_m_s2', positive=True)
s_d[g_m_s2] = 9.8
g_m_s2

자중에 의한 분포하중

In [ ]:
w_N_m = -mw_kg_m * g_m_s2
w_N_m

#### 반력

In [ ]:
vf_w_eq = sy.Eq(R_A_N + R_D_N + P_B_N + w_N_m * L_AD_m, 0)
vf_w_eq

In [ ]:
moment_A_w_eq = sy.Eq(P_B_N * L_AB_m + R_D_N * L_AD_m + (w_N_m * L_AD_m)*(0.5 * L_AD_m), 0)
moment_A_w_eq

In [ ]:
R_w_sol_dict = sy.solve((vf_w_eq, moment_A_w_eq), (R_A_N, R_D_N))
R_w_sol_dict

#### 전단력선도

In [ ]:
V_AD_w_N = sy.Piecewise((R_A_N + w_N_m * x, x < L_AB_m), 
                        (R_A_N + P_B_N + w_N_m * x, x >= L_AB_m))
V_AD_w_N

In [ ]:
V_AD_w_N = V_AD_N + w_N_m * x

In [ ]:
v_plot_list = splot.plot(V_AD_N.subs(R_sol_dict).subs(s_d),
                         V_AD_w_N.subs(R_w_sol_dict).subs(s_d), 
                         (x, 0, L_AD_m.subs(s_d)),
                         show=False
                         )
# janpenguin, "sympy – changing graph color of plot()",butterflyofdream.wordpress.com, 2017 Feb 01, https://butterflyofdream.wordpress.com/2014/02/01/sympy-changing-the-graph-color-of-plot/
# SymPy Development Team, "sympy.plotting.plot.plot()", Plotting Function Reference, sympy documentation, 2017 July 27, http://docs.sympy.org/latest/modules/plotting.html#sympy.plotting.plot.plot
v_plot_list.ylabel = 'V(N)'
plot_list = v_plot_list

In [ ]:
plot_list[0].line_color = 'blue'
plot_list[1].line_color = 'orange'
plot_list.xlabel = 'x(m)'
plot_list.show()

In [ ]:
V_AD_w_N.subs(R_w_sol_dict).subs(s_d).subs(x, 0)

#### 굽힘모멘트선도

In [ ]:
M_AD_w_Nm = M_AD_Nm + 0.5 * w_N_m * x ** 2

In [ ]:
m_plot_list = splot.plot(M_AD_Nm.subs(R_sol_dict).subs(s_d), 
                         M_AD_w_Nm.subs(R_w_sol_dict).subs(s_d), 
                         (x, 0, x_D_m),
                         show=False
                        )
# janpenguin, "sympy – changing graph color of plot()",butterflyofdream.wordpress.com, 2017 Feb 01, https://butterflyofdream.wordpress.com/2014/02/01/sympy-changing-the-graph-color-of-plot/
# SymPy Development Team, "sympy.plotting.plot.plot()", Plotting Function Reference, sympy documentation, 2017 July 27, http://docs.sympy.org/latest/modules/plotting.html#sympy.plotting.plot.plot
m_plot_list.ylabel = 'M(Nm)'
plot_list = m_plot_list

In [ ]:
plot_list[0].line_color = 'blue'
plot_list[1].line_color = 'orange'
plot_list.xlabel = 'x(m)'
plot_list.show()

In [ ]:
M_max_Nm = M_AD_w_Nm.subs(x, L_AB_m)
M_max_Nm

In [ ]:
M_max_Nm.subs(R_w_sol_dict).subs(s_d)

#### 자중을 포함한 최대 응력

In [ ]:
sigma_max_Pa = M_max_Nm / section_modulus_m3
sigma_max_Pa

In [ ]:
s_max_Pa = float(sigma_max_Pa.subs(R_w_sol_dict).subs(s_d).subs(section_modulus_mm3, 547e3))
s_max_Pa

In [ ]:
disp.display(disp.Math('$$\\sigma_{max}=%g (MPa)$$' % (s_max_Pa * 1e-6)))

자중을 포함한 최대 응력이 허용 최대응력보다 작은지 확인.<br>Assert if maximum stress is smaller than allowable maximum stress.

In [ ]:
assert (abs(s_d[sigma_allowed_Pa]) > abs(s_max_Pa))